In [85]:
from scipy import io
import pprint
import numpy as np

In [86]:
class calculatePose:
    def __init__(self, viconDataPath: str, imuDataPath: str, imuParamPath: str) -> None:
        viconDataLoader = io.loadmat(viconDataPath)
        self.viconTS = viconDataLoader["ts"]
        self.viconData = viconDataLoader["rots"]  # Rotation Matrix with Z-Y-X Euler Angles Rotation

        imuDataLoader = io.loadmat(imuDataPath)
        self.IMUTS = imuDataLoader["ts"]
        pprint.pprint(self.IMUTS)
        self.IMUData = imuDataLoader["vals"] # [ax ay az wz wx wy]^T
        # pprint.pprint(self.IMUData)

        imuParamLoader = io.loadmat(imuParamPath)
        imuParams = imuParamLoader["IMUParams"]
        scaleParams, biasParams = imuParams

        sx, sy, sz = scaleParams
        bax, bay, baz = biasParams

        wxTotal = 0
        wyTotal = 0
        wzTotal = 0
        totalIterations = 300
        for i in range(totalIterations):
            imuData = [
                self.IMUData[3, i],
                self.IMUData[4, i],
                self.IMUData[5, i],
            ]
            wz, wx, wy = imuData
            wxTotal += wx
            wyTotal += wy
            wzTotal += wz
        bgx = wxTotal / totalIterations
        bgy = wyTotal / totalIterations
        bgz = wzTotal / totalIterations

        metricIMUData = self.convertRawValuesToSIUnits(
            bax, bay, baz, sx, sy, sz, bgx, bgy, bgz
        )
        pprint.pprint(metricIMUData[0])

    def convertRawValuesToSIUnits(self, bax, bay, baz, sx, sy, sz, bgx, bgy, bgz):
        adjustedIMUData = np.zeros(
            (len(self.IMUData[0]), 6), dtype=float
        )  # t,ax,zy,az, wx,wy,wz
        for i in range(len(self.IMUData[0])):
            imuData = [
                self.IMUData[0, i],
                self.IMUData[1, i],
                self.IMUData[2, i],
                self.IMUData[3, i],
                self.IMUData[4, i],
                self.IMUData[5, i],
            ]
            # pprint.pprint(imuData)
            ax, ay, az, wz, wx, wy = imuData
            tilde_ax = (ax + bax) / sx
            tilde_ay = (ay + bay) / sy
            tilde_az = (az + baz) / sz

            tilde_wx = (3300 / 1023) * (np.pi / 180) * 0.3 * (wx - bgx)
            tilde_wy = (3300 / 1023) * (np.pi / 180) * 0.3 * (wy - bgx)
            tilde_wz = (3300 / 1023) * (np.pi / 180) * 0.3 * (wz - bgx)

            adjustedIMUData[i] = (
                tilde_ax,
                tilde_ay,
                tilde_az,
                tilde_wx,
                tilde_wy,
                tilde_wz,
            )
        return adjustedIMUData

    def alignData(self, imuTS, imuData, viconTS, viconData):
        pass

In [87]:
set1 = calculatePose(
    "Phase1/Data/Train/Vicon/viconRot1.mat",
    "Phase1/Data/Train/IMU/imuRaw1.mat",
    "Phase1/IMUParams.mat"
)

array([[1.29663678e+09, 1.29663678e+09, 1.29663678e+09, ...,
        1.29663684e+09, 1.29663684e+09, 1.29663684e+09]])
array([-5.48151123e+04, -5.35384197e+04,  6.72127665e+04,  7.20652078e-03,
        4.09870870e-02, -6.03546116e-02])
